In [1]:
%load_ext autoreload
%autoreload 2

from model import *



from torch import empty, cat, arange
from torch.nn.functional import fold, unfold
from torch import nn

In [2]:
x = torch.randn((1, 3, 4, 4))

# x = torch.tensor(
    # [[0, 1., 0],     
    # [2, 0, 0],
    # [0, 0, 1]])

# y = torch.ones(x.shape)
y = torch.ones((1, 3, 4, 4))

x, y

(tensor([[[[ 1.2890, -0.3293, -3.3882, -1.2922],
           [-0.1472, -1.4618,  0.8332, -0.6043],
           [-0.5990,  0.3278,  0.5609,  1.5212],
           [-0.5929, -1.3588, -1.9340, -0.0660]],
 
          [[-0.7778,  0.0927, -0.2413, -0.9580],
           [-0.2005, -0.7995,  1.7680,  1.2919],
           [-0.4552,  1.6358, -0.0690,  0.2430],
           [-1.1222,  0.0163, -0.1831, -0.4965]],
 
          [[-0.6739,  1.0487,  2.0044,  0.1517],
           [-0.8279, -0.4047, -0.9939,  1.2478],
           [-0.5730,  0.6923, -0.3886, -0.5263],
           [-0.4183, -2.1287,  0.2615,  2.5691]]]]),
 tensor([[[[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]],
 
          [[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]],
 
          [[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]]]]))

In [3]:
conv = nn.Conv2d(3, 4, 2)
expected = conv(y)
expected

tensor([[[[-0.2417, -0.2417, -0.2417],
          [-0.2417, -0.2417, -0.2417],
          [-0.2417, -0.2417, -0.2417]],

         [[ 0.2086,  0.2086,  0.2086],
          [ 0.2086,  0.2086,  0.2086],
          [ 0.2086,  0.2086,  0.2086]],

         [[-0.2186, -0.2186, -0.2186],
          [-0.2186, -0.2186, -0.2186],
          [-0.2186, -0.2186, -0.2186]],

         [[ 0.7873,  0.7873,  0.7873],
          [ 0.7873,  0.7873,  0.7873],
          [ 0.7873,  0.7873,  0.7873]]]])

In [4]:
# torch.nn.Unfold(kernel_size=2)(x)

kernel_size = (2, 2)
out_channels = 4

# Output of convolution as a matrix product
unfolded = torch.nn.functional.unfold(y, kernel_size=kernel_size)
# unfolded

wxb = conv.weight.view(out_channels, -1) @ unfolded + conv.bias.view(1, -1, 1)
actual = wxb.view(1, out_channels, y.shape[2] - kernel_size[0] + 1, y.shape[3] - kernel_size[1]+ 1)
torch.testing.assert_allclose(actual, expected)

In [60]:
(conv.weight.view(out_channels, -1) @ unfolded).shape

torch.Size([1, 4, 9])

In [59]:
conv.bias.shape

torch.Size([4])

In [39]:
conv.weight.view(out_channels, -1).shape

torch.Size([4, 12])

In [17]:
actual.shape

torch.Size([1, 4, 3, 3])

### Testing our Conv2d Layer 

Forward pass

In [130]:
# testing our conv2d layer
my_conv = Conv2d(3, 4, 2)
x2 = my_conv.forward(x)
x2.shape

torch.Size([1, 4, 3, 3])

In [131]:
torch.testing.assert_allclose(my_conv.forward(x), torch.nn.functional.conv2d(x, my_conv.weight, my_conv.bias))

Backward pass

### Testing our TransposeConv2d layer

Forward pass

In [132]:
my_t_conv = TransposeConv2d(3, 5, 2)
y_expected = my_t_conv.forward(x)
y_expected.shape

torch.Size([1, 5, 5, 5])

In [133]:
torch.testing.assert_allclose(my_t_conv.forward(x), torch.nn.functional.conv_transpose2d(x, my_t_conv.weight, my_t_conv.bias))

Backward pass

### Other tests

In [5]:
conv.weight.view(out_channels, -1).size()

torch.Size([4, 12])

In [118]:
x.size()

torch.Size([1, 3, 4, 4])

In [112]:
unfolded.size()

torch.Size([1, 12, 9])

In [115]:

wxb.size()

torch.Size([1, 4, 9])

In [81]:
# relu = ReLU()
f=Sigmoid()

f.forward(x)

tensor([[0.5000, 0.7311, 0.5000],
        [0.8808, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.7311]])

In [82]:
f.backward(y)

tensor([[0.5000, 0.2689, 0.5000],
        [0.1192, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.2689]])